In [1]:
import os
import cv2
from shutil import copyfile
import matplotlib.pyplot as plt
import csv
import random

In [2]:
root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_unity_new/objects/"
box_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_unity_new/unity_boxes/"
fcn_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/"
obj_list = os.listdir(root)

In [6]:
os.listdir(box_root)

['box_108', 'box_116', 'box_72']

In [179]:
for obj in obj_list:
    print(obj)
    box_type = None
    if '72' in obj:
        box_type = 'box_72'
    elif '108' in obj:
        box_type = 'box_108'
    elif '116' in obj:
        box_type = 'box_116'
    scene_list = os.listdir(os.path.join(root, obj))
    for scene in scene_list:
        data_list = os.listdir(os.path.join(root, obj, scene))
        for data in data_list:
            if "original" in data:
                original_path = os.path.join(root, obj, scene, data)
                seg_name = data.split("original")
                seg_name = seg_name[0] + "seg" + seg_name[1]
                seg_path = os.path.join(root, obj, scene, seg_name)
                box_path = os.path.join(box_root, box_type, scene, data)
                # copyfile(src, dst)
                copyfile(original_path, os.path.join(fcn_root, "image", obj+"_"+scene+"_"+data))
                copyfile(seg_path, os.path.join(fcn_root, "seg", obj+"_"+scene+"_"+seg_name))
                copyfile(box_path, os.path.join(fcn_root, "box", obj+"_"+scene+"_"+data))
                if box_type is None:
                    print('0')
                if not os.path.isfile(original_path):
                    print('1')
                if not os.path.isfile(seg_path):
                    print('2')
                if not os.path.isfile(box_path):
                    print('3')

3m_116
andes_108
cocacola_116
crayola_108
folgers_116
heineken_116
hunts_116
kellogg_108
kleenex_108
kotex_108
libava_72
macadamia_108
milo_108
mm_72
pocky_108
raisins_108
stax_116
swiss_miss_108
vanish_116
viva_116


In [4]:
seg_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/seg/"
mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/mask/"
seg_list = os.listdir(seg_root)
for seg in seg_list:
    obj_class = seg.split('_Scene')[0]
    if obj_class in obj_list:
        class_idx = obj_list.index(obj_class) + 1
        seg_path = os.path.join(seg_root, seg)
        img = cv2.imread(seg_path)
        img[img > 0] = class_idx
        img = img[:,:,2]
        cv2.imwrite(mask_root + seg, img)
    else:
        print('wrong')

# Generate csv file

In [181]:
image_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/image/"
mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/mask/"
image_list = os.listdir(image_root)
random.shuffle(image_list)
ratio = 0.9
training_list = []
test_list = []
count = 0
for image in image_list:
    if count < len(image_list)*ratio:
        training_list.append(image)
    else:
        test_list.append(image)
    count = count + 1

In [182]:
with open(fcn_root + 'train.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for image in training_list:
        mask_name = image.split("_original.png")[0] + "_seg.png"
        writer.writerow(["image/" + image, "mask/" + mask_name])
with open(fcn_root + 'test.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for image in test_list:
        mask_name = image.split("_original.png")[0] + "_seg.png"
        writer.writerow(["image/" + image, "mask/" + mask_name])